In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
features = pd.read_csv('features.csv')
features.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [3]:
stores = pd.read_csv('stores.csv')
stores.head(2)

,Store,Type,Size
0,1,A,151315
1,2,A,202307


In [4]:
train = pd.read_csv('trains.csv')
train.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True


In [5]:
train['Store'].nunique()

45

In [6]:
len(train['Store'])

421570

In [7]:
len(stores['Store'])

45

In [8]:
stores['Store'].nunique()

45

In [9]:
test = pd.read_csv('test.csv')
test.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [10]:
data = train.merge(stores, how='left', on='Store')

In [11]:
data.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315


In [12]:
len(data['Store'])

421570

In [13]:
data.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Type            0
Size            0
dtype: int64

In [14]:
features['Store'].nunique()

45

In [15]:
len(features['Store'])

8190

In [16]:
features.columns

Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday'],
      dtype='object')

In [17]:
data.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Type', 'Size'], dtype='object')

In [18]:
features.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [19]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [20]:
data = data.merge(features, how='left', on=['Store', 'Date'])

In [21]:
len(data)

421570

In [22]:
data.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday_x   421570 non-null  bool   
 5   Type          421570 non-null  object 
 6   Size          421570 non-null  int64  
 7   Temperature   421570 non-null  float64
 8   Fuel_Price    421570 non-null  float64
 9   MarkDown1     150681 non-null  float64
 10  MarkDown2     111248 non-null  float64
 11  MarkDown3     137091 non-null  float64
 12  MarkDown4     134967 non-null  float64
 13  MarkDown5     151432 non-null  float64
 14  CPI           421570 non-null  float64
 15  Unemployment  421570 non-null  float64
 16  IsHoliday_y   421570 non-null  bool   
dtypes: bool(2), float64(10), int64(3), object(2)
mem

In [24]:
data['Date'] = pd.to_datetime(data['Date'])

In [25]:
data = data.sort_values(by='Date')

In [26]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
277665,29,5,2010-02-05,15552.08,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
277808,29,6,2010-02-05,3200.22,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
277951,29,7,2010-02-05,10820.05,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False
278094,29,8,2010-02-05,20055.64,False,B,93638,24.36,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,10.064,False


In [27]:
data.tail()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
379648,40,87,2012-10-26,24638.96,False,A,155083,49.65,3.917,3605.71,55.98,0.28,486.81,1389.06,138.728161,4.145,False
181170,19,30,2012-10-26,3740.12,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181313,19,31,2012-10-26,3128.17,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
181599,19,33,2012-10-26,5740.14,False,A,203819,56.49,4.071,5430.75,90.07,NaN,904.34,1665.77,138.728161,7.992,False
421569,45,98,2012-10-26,1076.80,False,B,118221,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,False


In [28]:
data.isnull().sum().sum()

1422431

In [29]:
data.isnull().sum()/len(data)*100

Store            0.000000
Dept             0.000000
Date             0.000000
Weekly_Sales     0.000000
IsHoliday_x      0.000000
Type             0.000000
Size             0.000000
Temperature      0.000000
Fuel_Price       0.000000
MarkDown1       64.257181
MarkDown2       73.611025
MarkDown3       67.480845
MarkDown4       67.984676
MarkDown5       64.079038
CPI              0.000000
Unemployment     0.000000
IsHoliday_y      0.000000
dtype: float64

In [30]:
numerical_columns = data.select_dtypes(include=[np.number]).columns
numerical_columns

Index(['Store', 'Dept', 'Weekly_Sales', 'Size', 'Temperature', 'Fuel_Price',
       'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment'],
      dtype='object')

In [31]:
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].median())

In [32]:
data.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Type            0
Size            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday_y     0
dtype: int64

In [33]:
data = data.fillna(data.mode().iloc[0])

In [34]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Week'] = data['Date'].dt.isocalendar().week
data['DayOfWeek'] = data['Date'].dt.dayofweek

In [35]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Year,Month,Day,Week,DayOfWeek
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,5347.45,...,1481.31,3359.45,211.096358,8.106,False,2010,2,5,5,4
277665,29,5,2010-02-05,15552.08,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
277808,29,6,2010-02-05,3200.22,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
277951,29,7,2010-02-05,10820.05,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4
278094,29,8,2010-02-05,20055.64,False,B,93638,24.36,2.788,5347.45,...,1481.31,3359.45,131.527903,10.064,False,2010,2,5,5,4


In [36]:
x = data.drop(columns = ['Date', 'Weekly_Sales'])
y = data['Weekly_Sales']

In [37]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
from sklearn.preprocessing import OneHotEncoder

In [39]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [40]:
X_encoded = encoder.fit_transform(x[["Type"]])  # double brackets ensure 2D


In [41]:
print(X_encoded.shape)  # should match number of categories
print(encoder.get_feature_names_out())  # column names


(421570, 3)
['Type_A' 'Type_B' 'Type_C']


In [42]:
encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(["Type"]))


In [43]:
print(encoded_df)

        Type_A  Type_B  Type_C
0          1.0     0.0     0.0
1          0.0     1.0     0.0
2          0.0     1.0     0.0
3          0.0     1.0     0.0
4          0.0     1.0     0.0
...        ...     ...     ...
421565     1.0     0.0     0.0
421566     1.0     0.0     0.0
421567     1.0     0.0     0.0
421568     1.0     0.0     0.0
421569     0.0     1.0     0.0

[421570 rows x 3 columns]


In [44]:
x_numeric = x.drop(columns=["Type"])
x_final = pd.concat([x_numeric.reset_index(drop=True), encoded_df], axis=1)

In [45]:
x_final

,Store,Dept,IsHoliday_x,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Unemployment,IsHoliday_y,Year,Month,Day,Week,DayOfWeek,Type_A,Type_B,Type_C
0,1,1,False,151315,42.31,2.572,5347.45,192.00,24.60,1481.31,...,8.106,False,2010,2,5,5,4,1.0,0.0,0.0
1,29,5,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,10.064,False,2010,2,5,5,4,0.0,1.0,0.0
2,29,6,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,10.064,False,2010,2,5,5,4,0.0,1.0,0.0
3,29,7,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,10.064,False,2010,2,5,5,4,0.0,1.0,0.0
4,29,8,False,93638,24.36,2.788,5347.45,192.00,24.60,1481.31,...,10.064,False,2010,2,5,5,4,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,40,87,False,155083,49.65,3.917,3605.71,55.98,0.28,486.81,...,4.145,False,2012,10,26,43,4,1.0,0.0,0.0
421566,19,30,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,7.992,False,2012,10,26,43,4,1.0,0.0,0.0
421567,19,31,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,7.992,False,2012,10,26,43,4,1.0,0.0,0.0
421568,19,33,False,203819,56.49,4.071,5430.75,90.07,24.60,904.34,...,7.992,False,2012,10,26,43,4,1.0,0.0,0.0


In [46]:
import pickle

In [47]:
pickle.dump(encoder, open("encoder.pkl", "wb"))

In [48]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_final)

In [49]:
x_scaled

array([[-1.65819926, -1.41874236, -0.27510614, ...,  0.97797803,
        -0.79593806, -0.3352628 ],
       [ 0.53181888, -1.28756049, -0.27510614, ..., -1.02251786,
         1.25637918, -0.3352628 ],
       [ 0.53181888, -1.25476502, -0.27510614, ..., -1.02251786,
         1.25637918, -0.3352628 ],
       ...,
       [-0.25033046, -0.43487832, -0.27510614, ...,  0.97797803,
        -0.79593806, -0.3352628 ],
       [-0.25033046, -0.36928738, -0.27510614, ...,  0.97797803,
        -0.79593806, -0.3352628 ],
       [ 1.78325781,  1.76241805, -0.27510614, ..., -1.02251786,
         1.25637918, -0.3352628 ]])

In [50]:
pickle.dump(scaler, open("scaler.pkl", "wb"))

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)


In [52]:
model = RandomForestRegressor(n_estimators=100)
model.fit(x_train, y_train)

RandomForestRegressor()

In [53]:
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [54]:
print(f"Mean Squared Error : {mse}")
print(f"R-Squared: {r2}")

Mean Squared Error : 11683853.527200062
R-Squared: 0.9775310837647957


In [55]:
print(x["Dept"].unique())
print(x["Store"].unique())
print(x["Year"].unique())

[ 1  5  6  7  8 49  9 10 11 12 13 14 16 17 48 18 20 21  4  3  2 80 81 82
 83 85 87 54 90 22 91 93 94 95 96 97 52 51 98 92 79 23 25 55 56 58 42 59
 60 67 71 72 74 41 46 44 26 27 28 29 30 31 45 24 32 34 35 36 38 40 33 19
 37 78 47 99 50 65 43 39 77]
[ 1 29  2 28 27 31 32 30 24  3 25 23 26 39 40 38 37 41 42 34 33 36 35 11
  4 12 10 13 14  6  5  7  9  8 20 19 18 22 21 15 16 17 44 45 43]
[2010 2011 2012]


In [56]:
import joblib
joblib.dump(model, "model.joblib")

['model.joblib']

In [57]:
print(x.columns)

Index(['Store', 'Dept', 'IsHoliday_x', 'Type', 'Size', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Year', 'Month',
       'Day', 'Week', 'DayOfWeek'],
      dtype='object')
